# Write a script!

Part of being a mathematician, engineer, or coder is knowing how to be lazy and
cut corners! Let's build a script for writing tedious hard coded HDL files. 

In [4]:
# Define the three sentences
sentences = [
    "Hello, Fudan!",
    "Jimmy Wang or",
    "Jun Wei Wang",
    "From school of Software!"
]

def generate_verilog_block(num, sentence, label):
    """
    Generates a Verilog case block for a given sentence.
    For each character:
      - Compute its mapped hex value: (ord(character) - 0x20)
      - The case index is printed in hex (e.g., 'h0, 'h1, ...).
      - A comment shows the original character (using "space" for a space).
    """
    lines = []
    # Print a comment header for the sentence block.
    lines.append(f"// {label}: {sentence}")
    lines.append("always @(cnt_lcd)")
    lines.append("   case(cnt_lcd[5:1])")
    for i, ch in enumerate(sentence):
        # Subtract 0x20 from the ASCII value
        mapped_value = ord(ch) - 0x20
        # Format the computed value as a two-digit hexadecimal (uppercase)
        mapped_hex = f"{mapped_value:02X}"
        # The case index, in hex. For example, index 10 is 'hA.
        index_hex = f"{i:X}"
        # For clarity in the comment, use "space" if the character is a space.
        comment = "space" if ch == " " else ch
        lines.append(f"   'h{index_hex} : tmp{num} = 'h{mapped_hex};    // {comment}")
    lines.append("   default : tmp1 = 'h00;")
    lines.append("   endcase\n")
    return "\n".join(lines)

# Generate verilog blocks for all sentences
verilog_output = []
i = 0
for sentence in sentences:
    # Use a label to identify the sentence (optional)
    block_label = f"Sentence '{sentence}'"
    verilog_output.append(generate_verilog_block(i + 1, sentence, block_label))
    i = i + 1

# Join all blocks and print the final verilog code.
final_verilog = "\n".join(verilog_output)
print(final_verilog)


// Sentence 'Hello, Fudan!': Hello, Fudan!
always @(cnt_lcd)
   case(cnt_lcd[5:1])
   'h0 : tmp1 = 'h28;    // H
   'h1 : tmp1 = 'h45;    // e
   'h2 : tmp1 = 'h4C;    // l
   'h3 : tmp1 = 'h4C;    // l
   'h4 : tmp1 = 'h4F;    // o
   'h5 : tmp1 = 'h0C;    // ,
   'h6 : tmp1 = 'h00;    // space
   'h7 : tmp1 = 'h26;    // F
   'h8 : tmp1 = 'h55;    // u
   'h9 : tmp1 = 'h44;    // d
   'hA : tmp1 = 'h41;    // a
   'hB : tmp1 = 'h4E;    // n
   'hC : tmp1 = 'h01;    // !
   default : tmp1 = 'h00;
   endcase

// Sentence 'Jimmy Wang or': Jimmy Wang or
always @(cnt_lcd)
   case(cnt_lcd[5:1])
   'h0 : tmp2 = 'h2A;    // J
   'h1 : tmp2 = 'h49;    // i
   'h2 : tmp2 = 'h4D;    // m
   'h3 : tmp2 = 'h4D;    // m
   'h4 : tmp2 = 'h59;    // y
   'h5 : tmp2 = 'h00;    // space
   'h6 : tmp2 = 'h37;    // W
   'h7 : tmp2 = 'h41;    // a
   'h8 : tmp2 = 'h4E;    // n
   'h9 : tmp2 = 'h47;    // g
   'hA : tmp2 = 'h00;    // space
   'hB : tmp2 = 'h4F;    // o
   'hC : tmp2 = 'h52;    // r
   d

In [ ]:
# List of sentences to output.
sentences = [
    "Hello, Fudan!",
    "I am CHEN ZI KUN",
    "21307130004",
]

def verilog_case_block(sentence, tmp_name):
    """
    Generate a Verilog always block with a case statement that maps
    cnt_lcd[5:1] to the mapped hex values for each character in the sentence.
    The mapping subtracts 0x20 from the ASCII code.
    
    Parameters:
      sentence: the string to map.
      tmp_name: the Verilog temporary signal name (e.g. "tmp0").
      
    Returns a string containing the Verilog code.
    """
    lines = []
    lines.append(f"// Case block for {tmp_name} representing \"{sentence}\"")
    lines.append("always @(cnt_lcd)")
    lines.append("   case(cnt_lcd[5:1])")
    for i, ch in enumerate(sentence):
        # Compute mapped value by subtracting 0x20 from the ASCII value.
        mapped_val = ord(ch) - 0x20
        mapped_hex = f"{mapped_val:02X}"
        index_hex = f"{i:X}"  # index in hex (e.g. 0, 1, 2, ... in hex)
        # Use "space" as comment if the character is a space.
        comment = "space" if ch == " " else ch
        lines.append(f"   'h{index_hex} : {tmp_name} = 'h{mapped_hex};    // {comment}")
    lines.append("   default : " + tmp_name + " = 'h00;")
    lines.append("   endcase")
    return "\n".join(lines)

# Generate a case block for each sentence.
verilog_sentences = []
for idx, sentence in enumerate(sentences):
    tmp_name = f"tmp{idx}"
    verilog_sentences.append(verilog_case_block(sentence, tmp_name))
    verilog_sentences.append("")  # Blank line for separation

# Generate the multiplexer block that chooses which sentence to output.
mux_lines = []
mux_lines.append("// Multiplexer to select the active sentence output")
mux_lines.append("always @(posedge clk or posedge rst)")
mux_lines.append("   if (rst)")
mux_lines.append("       lcd_db <= 0;")
mux_lines.append("   else if (lcd_en)")
mux_lines.append("       case(sentence_sel)")
for idx in range(len(sentences)):
    mux_lines.append(f"           {idx} : lcd_db <= tmp{idx};")
mux_lines.append("           default : lcd_db <= 0;")
mux_lines.append("       endcase")
mux_lines.append("   else")
mux_lines.append("       lcd_db <= 0;")

# Generate the clocked counter block that increments cnt_lcd and the sentence selector.
counter_lines = []
counter_lines.append("// Counter and sentence selector update block")
counter_lines.append("always @(posedge clk or posedge rst)")
counter_lines.append("   if (rst) begin")
counter_lines.append("       cnt_lcd <= 0;")
counter_lines.append("       sentence_sel <= 0;")
counter_lines.append("   end else begin")
counter_lines.append("       cnt_lcd <= cnt_lcd + 1;")
counter_lines.append("       // When cnt_lcd reaches a maximum, increment sentence_sel")
counter_lines.append("       if ((cnt_lcd & 6'b111111) == 6'b111111) begin")
counter_lines.append(f"           if (sentence_sel < {len(sentences)})")
counter_lines.append("               sentence_sel <= sentence_sel + 1;")
counter_lines.append("           else")
counter_lines.append("               sentence_sel <= 0;")
counter_lines.append("       end")
counter_lines.append("   end")

# Combine all parts into final Verilog output.
final_verilog = "\n\n".join([
    "// Generated Verilog Code",
    "\n".join(verilog_sentences),
    "\n".join(mux_lines),
    "\n".join(counter_lines)
])

print(final_verilog)


// Generated Verilog Code

// Case block for tmp0 representing "Hello, Fudan!"
always @(cnt_lcd)
   case(cnt_lcd[5:1])
   'h0 : tmp0 = 'h28;    // H
   'h1 : tmp0 = 'h45;    // e
   'h2 : tmp0 = 'h4C;    // l
   'h3 : tmp0 = 'h4C;    // l
   'h4 : tmp0 = 'h4F;    // o
   'h5 : tmp0 = 'h0C;    // ,
   'h6 : tmp0 = 'h00;    // space
   'h7 : tmp0 = 'h26;    // F
   'h8 : tmp0 = 'h55;    // u
   'h9 : tmp0 = 'h44;    // d
   'hA : tmp0 = 'h41;    // a
   'hB : tmp0 = 'h4E;    // n
   'hC : tmp0 = 'h01;    // !
   default : tmp0 = 'h00;
   endcase

// Case block for tmp1 representing "My name is Jimmy Wang"
always @(cnt_lcd)
   case(cnt_lcd[5:1])
   'h0 : tmp1 = 'h2D;    // M
   'h1 : tmp1 = 'h59;    // y
   'h2 : tmp1 = 'h00;    // space
   'h3 : tmp1 = 'h4E;    // n
   'h4 : tmp1 = 'h41;    // a
   'h5 : tmp1 = 'h4D;    // m
   'h6 : tmp1 = 'h45;    // e
   'h7 : tmp1 = 'h00;    // space
   'h8 : tmp1 = 'h49;    // i
   'h9 : tmp1 = 'h53;    // s
   'hA : tmp1 = 'h00;    // space
   'hB 